# Import libraries and datasets

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
!pip install -q kaggle opencv-python matplotlib numpy torchvision torch
from google.colab import files
files.upload()  # Upload your kaggle.json file here

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kacpergregorowicz/house-plant-species
!unzip -q house-plant-species.zip -d plant_data

Dataset URL: https://www.kaggle.com/datasets/kacpergregorowicz/house-plant-species
License(s): CC-BY-NC-SA-4.0
100% 4.85G/4.85G [03:51<00:00, 23.2MB/s]
100% 4.85G/4.85G [03:51<00:00, 22.5MB/s]


# Set Parameters and initialise CNN model

In [ ]:
DATA_PATH = "/content/plant_data/house_plant_species"
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 30
NUM_CLASSES = 47
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomRotation(40),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = datasets.ImageFolder(DATA_PATH, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [ ]:
class PlantClassifier(nn.Module):
    def __init__(self, num_classes):
        super(PlantClassifier, self).__init__()
        self.base_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.last_channel, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.base_model(x)

model = PlantClassifier(NUM_CLASSES).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train Model

😵‍💫...didnt test validation accuracy...but its pretty much accurate

In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

train_model(model, train_loader, criterion, optimizer, EPOCHS)

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/30: Train Loss: 1.8916, Train Acc: 0.5239


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 2/30: Train Loss: 0.6546, Train Acc: 0.8160


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch 8/30: Train Loss: 0.1830, Train Acc: 0.9438


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 9/30: Train Loss: 0.1537, Train Acc: 0.9526


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


KeyboardInterrupt: 

# Save Model

In [ ]:
torch.save(model.state_dict(), 'plant_classification_model.pth')

# Convert to H5 format
import h5py
with h5py.File("plant_classification_model.h5", "w") as h5f:
    for name, param in model.state_dict().items():
        h5f.create_dataset(name, data=param.cpu().numpy())

print("Training completed and model saved in H5 format!")

Training completed and model saved in H5 format!


# Test Model

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import h5py
import numpy as np
import pickle
from torchvision import models
import os

# Step 1: Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PlantClassifier(torch.nn.Module):
    def __init__(self, num_classes):
        super(PlantClassifier, self).__init__()
        self.base_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        self.base_model.classifier = torch.nn.Sequential(
            torch.nn.Linear(self.base_model.last_channel, 1024),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.base_model(x)

# Step 2: Read folder names and create pickle file
def create_class_names_pickle(data_path):
    class_names = sorted(os.listdir(data_path))
    with open("class_names.pkl", "wb") as f:
        pickle.dump(class_names, f)
    return class_names

DATA_PATH = "/content/plant_data/house_plant_species"
if not os.path.exists("class_names.pkl"):
    class_names = create_class_names_pickle(DATA_PATH)
else:
    with open("class_names.pkl", "rb") as f:
        class_names = pickle.load(f)

NUM_CLASSES = len(class_names)

# Initialize model
model = PlantClassifier(NUM_CLASSES).to(device)

# Load weights from h5 file
with h5py.File("plant_classification_model.h5", "r") as h5f:
    state_dict = {name: torch.tensor(np.array(h5f[name])) for name in h5f.keys()}
    model.load_state_dict(state_dict)

model.eval()

# Step 3: Define Image Preprocessing
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device)

# Step 4: Model Prediction
def predict(image_path):
    image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item()
    return class_names[predicted_class]

# Test the model with an example image
test_image_path = "/content/plant_data/house_plant_species/Aloe Vera/107.jpg"
if os.path.exists(test_image_path):
    prediction = predict(test_image_path)
    print(f"Predicted class: {prediction}")
else:
    print("Test image not found. Please provide a valid path.")

Predicted class: Aloe Vera
